# Tutorial 3: Clipper in Action with Pong

We have already explored some of the features of Clipper (insert recap here). Now let's take a look at Clipper in action with Pong! Released by Atari in 1972, Pong was the first commercially successful video game. You can read more about it <a href="https://en.wikipedia.org/wiki/Pong">here</a>

The goal of this tutorial is to use Clipper to deploy several AI policies to play against, and in doing so explore:
(list of Clipper features we want to highlight)

This tutorial will be broken up into 3 main parts:
#### 1. Starting Clipper and Deploying First Policy
#### 2. Training a Better Model
#### 3. Deploying an updated, trained model

## Part 1: Starting Clipper and Deploying First Policy

Install packaged env from requirements.txt (not sure if we should do this before opening jupyter notebook or not)

In [1]:
#Import dependencies
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as py_deployer
import random
import numpy as np
import pandas as pd
from datascience import *
from sklearn import linear_model
from IPython.display import Image
from IPython.core.display import HTML 


Our first step is to get Clipper started and deploy our first AI policy: one that guesses which direction to move the paddle randomly. First, start Docker (or make sure it is running)

In [2]:
#Starting Clipper, make sure you have Docker running before you run this cell
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

18-10-02:00:23:05 INFO     [clipper_admin.py:1278] [default-cluster] Stopped all Clipper cluster and all model containers
18-10-02:00:23:05 INFO     [docker_container_manager.py:151] [default-cluster] Starting managed Redis instance in Docker
18-10-02:00:23:08 INFO     [docker_container_manager.py:229] [default-cluster] Metric Configuration Saved at /private/var/folders/2f/f3nxmw0j3_1d57n7fs8_nklw0000gp/T/tmpte5lmhfx.yml
18-10-02:00:23:09 INFO     [clipper_admin.py:138] [default-cluster] Clipper is running


Running the cell below will register an application in Clipper called "pong" and create a Clipper endpoint for our random policy at http://localhost:1337/pong/predict

In [3]:
def random_predict(xs):
    action = random.randint(0, 2)
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)

18-10-02:00:23:14 INFO     [clipper_admin.py:215] [default-cluster] Application pong was successfully registered
18-10-02:00:23:14 INFO     [deployer_utils.py:41] Saving function to /var/folders/2f/f3nxmw0j3_1d57n7fs8_nklw0000gp/T/tmp1ke4ds06clipper
18-10-02:00:23:14 INFO     [deployer_utils.py:51] Serialized and supplied predict function
18-10-02:00:23:14 INFO     [python.py:192] Python closure saved
18-10-02:00:23:14 INFO     [python.py:202] Using Python 3.5 base image
18-10-02:00:23:14 INFO     [clipper_admin.py:467] [default-cluster] Building model Docker image with model data from /var/folders/2f/f3nxmw0j3_1d57n7fs8_nklw0000gp/T/tmp1ke4ds06clipper
18-10-02:00:23:59 INFO     [clipper_admin.py:472] [default-cluster] Step 1/2 : FROM clipper/python35-closure-container:develop
18-10-02:00:23:59 INFO     [clipper_admin.py:472] [default-cluster]  ---> f7d5f86ed257
18-10-02:00:23:59 INFO     [clipper_admin.py:472] [default-cluster] Step 2/2 : COPY /var/folders/2f/f3nxmw0j3_1d57n7fs8_nklw0

Now that we have a model deployed, let's see how it works! Run the cell below to start the game

In [4]:
!python pong-server.py localhost:1337

18-10-02:00:24:46 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
18-10-02:00:24:51 INFO     [pong-server.py:51] Local path: /
127.0.0.1 - - [02/Oct/2018 00:24:51] code 403, message Forbidden
127.0.0.1 - - [02/Oct/2018 00:24:51] "GET / HTTP/1.1" 403 -
18-10-02:00:24:53 INFO     [pong-server.py:51] Local path: /favicon.ico
127.0.0.1 - - [02/Oct/2018 00:24:53] code 403, message Forbidden
127.0.0.1 - - [02/Oct/2018 00:24:53] "GET /favicon.ico HTTP/1.1" 403 -
18-10-02:00:24:59 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/index.html
127.0.0.1 - - [02/Oct/2018 00:24:59] "GET /pong/ HTTP/1.1" 200 -
18-10-02:00:24:59 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/pong.css
127.0.0.1 - - [02/Oct/2018 00:24:59] "GET /pong/pong.css HTTP/1.1" 200 -
18-10-02:00:24:59 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/game.js
127.0.0.1 -

Congratulations! We have depolyed our first model to Clipper! Let's take a look at it by clicking http://localhost:3000/pong/ and pressing 1 to start the game.

## Part 2: Training a better model

As you probably noticed, the random-guessing policy did not perform well at all. In order to train a better model, we are going to use imitation learning. Let's play the game again, this time with 2 actual players, and we will collect data on how you play the game to train our model! 

Go to http://localhost:3000/pong/ and press 2. Find someone next to you and play pong against them!!

<b>Part to collect data into a output.csv --> currently doing this in really hacky way to just get it to work, but trying to figure out better solution</b>

Now that we have our data, lets train a new model! First, run the cell below to clean the data and format it for scikit-learn's LogisticRegression model.

In [10]:
#Read data into table
data_table= Table.read_table('out.csv')

#Convert labels into numeric values
def convert_label(label):
    if(label=="down"):
        return 1
    elif(label=="up"):
        return 2
    else:
        return 0
data_table_2 = data_table.with_column("label", data_table.apply(convert_label, 0))

#Scale our data down
final_table = Table().with_columns("label", data_table.apply(convert_label, 0),
                                  "paddle_y", data_table.column(1)/500.0,
                                  "ball_x", data_table.column(2)/500.0,
                                  "ball_y", data_table.column(3)/500.0,
                                  "ball_dx", data_table.column(4)/500.0,
                                  "ball_dy", data_table.column(5)/500.0,
                                  "x_prev", data_table.column(6)/500.0,
                                  "y_prev", data_table.column(7)/500.0)
data_df = final_table.to_df()
data_df.take(np.arange(5))

,label,paddle_y,ball_x,ball_y,ball_dx,ball_dy,x_prev,y_prev
0,1,0.420000,0.105540,0.560353,0.319816,0.319816,0.089250,0.544063
1,1,0.657204,0.299980,0.754793,0.329400,0.329400,0.283859,0.738672
2,0,0.677400,0.316800,0.771613,0.330216,0.330216,0.299980,0.754793
3,2,0.677400,0.383163,0.837977,0.333416,0.333416,0.366513,0.821326
4,0,0.618000,0.433356,0.888169,0.335816,0.335816,0.416920,0.871733


We are going to use the data to train a scikit-learn Logistic Regression model. You can read more about the particular model <a url="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">here</a>

In [11]:
labels = data_df['label']
training_data= data_df.drop(['label'], axis=1)

model = linear_model.LogisticRegression()
model.fit(training_data, labels)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Part 3: Deploying updated model

Now that we have an updated model, we can deploy that model onto Clipper. Once the system realizes there is a new version of the model, it will automatically switch to serving the newer version of the model.

In [12]:
Image(url= "update_model.png")

In [13]:
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

py_deployer.deploy_python_closure(clipper_conn, name="pong", version=4, input_type="doubles", func=predict, pkgs_to_install=["ray", "numpy","scipy", "pandas", "datascience", "sklearn"])


18-10-02:00:35:29 INFO     [deployer_utils.py:41] Saving function to /var/folders/2f/f3nxmw0j3_1d57n7fs8_nklw0000gp/T/tmpzk1vtimiclipper
18-10-02:00:35:29 INFO     [deployer_utils.py:51] Serialized and supplied predict function
18-10-02:00:35:29 INFO     [python.py:192] Python closure saved
18-10-02:00:35:29 INFO     [python.py:202] Using Python 3.5 base image
18-10-02:00:35:29 INFO     [clipper_admin.py:467] [default-cluster] Building model Docker image with model data from /var/folders/2f/f3nxmw0j3_1d57n7fs8_nklw0000gp/T/tmpzk1vtimiclipper
18-10-02:00:35:30 INFO     [clipper_admin.py:472] [default-cluster] Step 1/3 : FROM clipper/python35-closure-container:develop
18-10-02:00:35:30 INFO     [clipper_admin.py:472] [default-cluster]  ---> f7d5f86ed257
18-10-02:00:35:30 INFO     [clipper_admin.py:472] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install ray numpy scipy pandas datascience sklearn
18-10-02:00:35:30 INFO     [clipper_admin.py:472] [default-clu

You may notice that it seems like we only deployed the predict function to Clipper. However, Clipper will track the dependencies that the model in the predict function uses and include them in the model Docker container as dependencies. You can take a look at the logs above to see the Docker container downloading said dependencies.

Let's run our server again (or refresh the page if you haven't stopped the server previously)

In [ ]:
!python pong-server.py localhost:1337

18-10-02:00:35:40 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
18-10-02:00:35:46 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/index.html
127.0.0.1 - - [02/Oct/2018 00:35:46] "GET /pong/ HTTP/1.1" 200 -
18-10-02:00:35:46 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/pong.css
127.0.0.1 - - [02/Oct/2018 00:35:46] "GET /pong/pong.css HTTP/1.1" 200 -
18-10-02:00:35:46 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/game.js
127.0.0.1 - - [02/Oct/2018 00:35:46] "GET /pong/game.js HTTP/1.1" 200 -
18-10-02:00:35:46 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/pong.js
127.0.0.1 - - [02/Oct/2018 00:35:46] "GET /pong/pong.js HTTP/1.1" 200 -
18-10-02:00:35:47 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/rl_and_pong/javascript-pong/static/images/press1.png
127.0.0

127.0.0.1 - - [02/Oct/2018 00:35:49] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":875,"output":0,"default":true,"default_explanation":"Failed to retrieve a prediction response within the specified latency SLO"}' in 114.46000000000001 ms
{'Content-Length': '140', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=12>
18-10-02:00:35:49 INFO     [pong-server.py:73] 140
140
18-10-02:00:35:49 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.480588, 0.18502243199999993, 0.46263672389817806, 0.32376799999999994, 0.32376799999999994, 0.16949999999999996, 0.44711429189817803]}
127.0.0.1 - - [02/Oct/2018 00:35:49] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":876,"output":0,"default":true,"default_explanation":"Failed to retrieve a prediction response within the specified latency SLO"}' in 136.343 ms
{'Content-Length': '140', 'Content-Type': 'application/js

<_io.BufferedReader name=8>
18-10-02:00:35:49 INFO     [pong-server.py:73] 134
134
18-10-02:00:35:49 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.580776, 0.3821632319999998, 0.659777523898178, 0.3333679999999999, 0.3333679999999999, 0.36518227199999986, 0.6427965638981781]}
<_io.BufferedReader name=10>
18-10-02:00:35:49 INFO     [pong-server.py:73] 144
144
18-10-02:00:35:49 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.6005759999999999, 0.39885163199999984, 0.6764659238981781, 0.3341679999999999, 0.3341679999999999, 0.3821632319999998, 0.659777523898178]}
127.0.0.1 - - [02/Oct/2018 00:35:49] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":890,"output":1,"default":false}' in 58.653999999999996 ms
{'Content-Length': '43', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
127.0.0.1 - - [02/Oct/2018 00:35:49] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '

<_io.BufferedReader name=8>
18-10-02:00:35:50 INFO     [pong-server.py:73] 131
131
18-10-02:00:35:50 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.816, 0.6363438719999998, 0.9139581638981781, 0.3453519999999998, 0.3453519999999998, 0.6194408319999998, 0.8970551238981781]}
127.0.0.1 - - [02/Oct/2018 00:35:50] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":905,"output":1,"default":false}' in 39.28 ms
{'Content-Length': '43', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:35:50 INFO     [pong-server.py:73] 119
119
18-10-02:00:35:50 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.816, 0.6543238079999998, 0.926, 0.3461839999999998, -0.3461839999999998, 0.6363438719999998, 0.9139581638981781]}
<_io.BufferedReader name=10>
18-10-02:00:35:50 INFO     [pong-server.py:73] 121
121
18-10-02:00:35:50 INFO     [pong-server.py:75] http://l

127.0.0.1 - - [02/Oct/2018 00:35:51] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":919,"output":2,"default":false}' in 35.78 ms
{'Content-Length': '43', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:35:51 INFO     [pong-server.py:73] 138
138
18-10-02:00:35:51 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.5784000000000002, 0.9005726079999996, 0.6803116479999999, 0.3573839999999998, -0.3573839999999998, 0.8827234079999996, 0.698120848]}
127.0.0.1 - - [02/Oct/2018 00:35:51] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":920,"output":2,"default":false}' in 19.116 ms
{'Content-Length': '43', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:35:51 INFO     [pong-server.py:73] 147
147
18-10-02:00:35:51 INFO     [pong-server.py:75] http://localhost:1337/pong/p

...

127.0.0.1 - - [02/Oct/2018 00:37:19] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2674,"output":0,"default":true,"default_explanation":"Failed to retrieve a prediction response within the specified latency SLO"}' in 119.94399999999999 ms
{'Content-Length': '141', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:37:19 INFO     [pong-server.py:73] 145
145
18-10-02:00:37:19 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4203960000000003, 1.2077856799999958, 0.10907650799999989, 0.4646159999999986, 0.2534839999999996, 1.184574879999996, 0.09642230799999991]}
127.0.0.1 - - [02/Oct/2018 00:37:19] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2675,"output":0,"default":false}' in 120.639 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=10>
18-10-02

<_io.BufferedReader name=8>
18-10-02:00:37:19 INFO     [pong-server.py:73] 148
148
18-10-02:00:37:19 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4203960000000003, 0.9857077280000004, 0.2782767719999997, -0.47507999999999856, 0.26394799999999957, 1.0094017280000005, 0.26509937199999967]}
127.0.0.1 - - [02/Oct/2018 00:37:19] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2690,"output":0,"default":false}' in 37.169000000000004 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:37:19 INFO     [pong-server.py:73] 145
145
18-10-02:00:37:19 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4203960000000003, 0.9633966400000004, 0.2906999999999996, -0.4758319999999986, 0.2646999999999996, 0.9857077280000004, 0.2782767719999997]}
<_io.BufferedReader name=10>
18-10-02:00:37:20 INFO     [pong-server.py:73] 147
14

127.0.0.1 - - [02/Oct/2018 00:37:20] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2703,"output":1,"default":false}' in 94.96300000000001 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:37:20 INFO     [pong-server.py:73] 147
147
18-10-02:00:37:20 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4603920000000003, 0.6269544320000016, 0.47990999999999917, -0.48703199999999847, 0.2758999999999995, 0.6517306400000015, 0.4658599079999992]}
127.0.0.1 - - [02/Oct/2018 00:37:20] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2704,"output":1,"default":false}' in 72.651 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=10>
18-10-02:00:37:20 INFO     [pong-server.py:73] 148
148
18-10-02:00:37:20 INFO     [pong-server.py:75] http

127.0.0.1 - - [02/Oct/2018 00:37:21] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2719,"output":0,"default":false}' in 22.387 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:37:21 INFO     [pong-server.py:73] 146
146
18-10-02:00:37:21 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.5993880000000004, 0.25827825600000265, 0.6910471079999988, -0.4990159999999983, 0.2878839999999993, 0.2826724160000026, 0.6769599999999988]}
127.0.0.1 - - [02/Oct/2018 00:37:21] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2720,"output":1,"default":false}' in 27.241 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:37:21 INFO     [pong-server.py:73] 148
148
18-10-02:00:37:21 INFO     [pong-server.py:75] http://localhost:

127.0.0.1 - - [02/Oct/2018 00:37:22] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2734,"output":0,"default":false}' in 19.271 ms
{'Content-Length': '44', 'Content-Type': 'application/json'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-10-02:00:37:22 INFO     [pong-server.py:73] 131
131
18-10-02:00:37:22 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.7589760000000003, 1.2062068, 0.1844823947605741, -0.31823199999999996, 0.31823199999999996, 1.222374208, 0.1682733707605741]}
<_io.BufferedReader name=10>
18-10-02:00:37:22 INFO     [pong-server.py:73] 139
139
18-10-02:00:37:22 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.7589760000000003, 1.1909500960000001, 0.19977596276057408, -0.31899999999999995, 0.31899999999999995, 1.2062068, 0.1844823947605741]}
127.0.0.1 - - [02/Oct/2018 00:37:22] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":2735,"o

Cool points here:
1. We never restarted the pong server, but our code seamlessly switches to serving the updated model

## 4. Conclusion

Just as a recap, here's what we did today:

1. Deployed an initial random policy to Clipper and served predictions
2. Trained a new model with data that you generated while playing pong
3. Deployed new version of model and seamlessly switch the model being served

By doing so, we've explored the following Clipper features:

1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation


In [10]:
!python stop_c.py


18-09-05:21:54:05 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers


# TO-DOS

1. Figure out a more seamless transition between training data and stuff
    - Node server
    - Somehow saving the console
    - other ideas appreciated
2. Fix copy on tutorial
3. More content(introduce rollbacks for older models?)
4. Time concerns